# Nichtlineare Optimierung mit Nebenbedingungen

## Dr. Anselm Hudde - 24.01.2022

In [ ]:
%run nonlinear_programming.py

## Worum geht es bei der nichtlinearen Optimierung?

Bei der Optimierung geht es darum, das Minimum (bzw. das Maximum) einer Funktion $f$ zu finden.

- Wie maximiert ein Unternehmen den Ertrag, unter Berücksichtigung der vorhandenen Ressourcen?

- Wie erreicht man mit einer Portfolioallokation eines maximale Rendite, wenn die erlaubte Volatilität beschränkt ist?

- Wie kann man die richtigen Modellparameter schätzen (statistische Modelle, neuronale Netze)?

## Beispiel aus der Portfoliooptimierung:

Ein Investor will $10\,000$ Euro Investieren, wobei diese auf zwei Aktien, $A$ und $B$, aufgeteilt werden.

- Erwartete jährliche Renditen: 

    Aktie $A$: $r_A = 7\%$

    Aktie $B$: $r_{A} = 9\%$
    
    
- Volatilitäten:

    Aktie $A$: $\sigma_A = 20\%$

    Aktie $B$: $\sigma_{B} = 30\%$
    
    
- Korrelation:

    $\rho_{A, B} = 0.7$

Der Investor möchte sein Geld so anlegen, dass er eine erwartete Rendite von $5\%$ erreicht, und dabei so wenig Volatilität wie möglich zulassen.
Die Volatilität eines Portfolios bestehen aus $x_1$ Euro in Aktie 1 und $x_2$ Euro in in Aktie 2 berechnet sich als

\begin{equation}
f (x_{1}, x_{2})
=
\sqrt{ \sigma_1^2 x_1^2 + \sigma_2^2 x_2^2 + 2 \sigma_1 x_1 \sigma_2 x_2 \rho_{1, 2}}
\end{equation}
wobei $\theta$ die Risikoaversion des Investor darstellt.

Unser Ziel ist also: 

\begin{equation}
\begin{split}
\text{Minimiere }& f(x) \\
\text{wobei }& x_1 + x_2 - 1 \leq 0 \\
\text{und }& \frac{r_1 * x_{1} + r_{2} * x_{2}}{10\,000} - 0.05 = 0 \text{ gilt.}
\end{split}
\end{equation}

Wenn wir

\begin{equation}
g(x_1, x_2) = x_1 + x_2 - 1.
\end{equation}
	
\begin{equation}
h(x_1, x_2) = \frac{r_1 * x_{1} + r_{2} * x_{2}}{10\,000}
\end{equation}

definieren haben wir auch eine allgemeine Defintion des Problems:

## Formulierung eines nichtlinearen Optimierungsproblems mit Nebenbedingungen

Sei $D \subseteq \mathbb{R}^n$ eine Teilmenge, und seien

\begin{equation}
\begin{split}
f \colon D \to \mathbb{R} \\
g \colon D \to \mathbb{R} \\
h \colon D \to \mathbb{R}
\end{split}
\end{equation}

drei (möglicherweise nichtlineare) Funktionen.
Das Ziel der nichtlinearen Optimierung:

\begin{equation}
\begin{split}
\text{Minimiere }& f(x) \\
\text{wobei }& g(x) \leq 0 \\
\text{und }& h(x) = 0 \text{ gilt.}
\end{split}
\end{equation}

## Beispielfunktion

Wir betrachten die Funktion

\begin{equation}f(x, y) = sin(x) + 0.05 \cdot x^2 + 0.1 \cdot y^2, \end{equation}

die wir minimieren wollen:

In [ ]:
import numpy as np
def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

surface_plot = plot()
surface_plot.plot_surface(-5,1,-3,3,f)
surface_plot.show()

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)
contour_plot.show()

## Das Gradientenverfahren: Idee

Die Idee ist es, immer in die Richtung zu gehen, in die es am steilsten bergab geht.
Doch wie messen wir die Steigung?
Hier kommt der **Gradient** ins Spiel:

**Definition:**
Der Gradient der Funktion $f \colon \mathbb{R}^{n} \to \mathbb{R}$ ist die Funktion

\begin{equation}
\nabla f \colon \mathbb{R}^{n} \to \mathbb{R}^{n};~~
x =
\begin{pmatrix}
x_{1} \\ \vdots \\ x_{n}
\end{pmatrix}
\mapsto
\begin{pmatrix}
\tfrac{\partial f}{x_{1}} (x) \\ \vdots \\ \tfrac{\partial f}{x_{n}} (x)
\end{pmatrix}.
\end{equation}

## **Anschauung:** Der Gradient zeigt immer in Richtung des stärksten Anstiegs.

Beispiel 1: $f(x, y) = x$, $\nabla f(x, y) = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$.

In [ ]:
surface_plot = plot()
surface_plot.plot_surface(-5,1,-3,3,function = lambda x: x[0])
surface_plot.show()

Beispiel 1: $f(x, y) = \tfrac{1}{3} (x+y)$, $\nabla f(x, y) = \begin{pmatrix} \tfrac{1}{3} \\ \tfrac{1}{3} \end{pmatrix}$.

In [ ]:
surface_plot = plot()
surface_plot.plot_surface(-5,1,-3,3,function = lambda x: (1/3)*(x[0]+x[1]))
surface_plot.show()

## Zurück zur eigentlichen Funktion:

Der Gradient von
$𝑓(𝑥,𝑦)= 𝑠𝑖𝑛(𝑥)+0.05\cdot 𝑥^2 + 0.1 \cdot 𝑦^2$:

\begin{equation}
\nabla f(x, y)
=
\begin{pmatrix}
\frac{\partial f}{\partial x} \\ \frac{\partial f}{\partial y} \end{pmatrix}
=
\begin{pmatrix}
\cos(x) + 0.1x \\ 0.2 y
\end{pmatrix}
\end{equation}

Beispiele: 
\begin{equation}
\nabla f(-4, -2) =
\begin{pmatrix}
-1.05 \\ -0.40
\end{pmatrix},
\end{equation}
\begin{equation}
\nabla f(-1, 2) =
\begin{pmatrix}
0.44 \\ 0.20
\end{pmatrix}.
\end{equation}


In [ ]:
def grad_f(x):
    return np.array([np.cos(x[0])+0.1*x[0], 0.2*x[1]])

contour_plot.add_gradients(grad_f)
contour_plot.show()

## Die Schritte beim Gradientenverfahren

Wir starten mit einem Punkt $x_0 \in \mathbb{R}^{2}$.
Dann gehen wir mit jeden Schritt in die Gegenrichtung des Gradienten
$\nabla f(x, y)$, und zwar multipliziert mit der Lernrate $\gamma_n$:

\begin{equation}
	x_{n+1}= x_{n} - \gamma_{n} \nabla f({x} _{n}),\ n\geq 0.
\end{equation}
Wenn die Lernrate dann klein genug ist, gilt
$f(x_{n+1}) \leq f(x_n)$, und das Verfahren konvergiert.

### Lernrate $\gamma = 1$:

Wir fangen im Punkt
$x_0 = \begin{pmatrix} -4 \\ -2 \end{pmatrix}$
an, und verwenden eine Lernrate von $\gamma = 1$.
Dann gilt

\begin{equation} \nabla f(x_0) =
\begin{pmatrix}\cos(-4)+0.1\cdot(-4) \\ 0.2\cdot(-2) \end{pmatrix}
=
\begin{pmatrix} -1.05 \\ -0.40 \end{pmatrix}
\end{equation}

und damit

\begin{equation}
x_{1} = x_{0} + \gamma_{0} \cdot \nabla f(x_0)
=
\begin{pmatrix} -4 \\ -2 \end{pmatrix}
- 1 \cdot 
\begin{pmatrix} -1.05 \\ -0.40 \end{pmatrix}
=
\begin{pmatrix}
-2.95 \\ -1.60
\end{pmatrix}.
\end{equation}
Die nächsten Schritte folgen analog:

$x_{2} = \begin{pmatrix} -1.67 \\ -1.28\end{pmatrix}, ~
x_{3} = \begin{pmatrix} -1.40 \\ -1.28\end{pmatrix}$

usw...

In [ ]:
# Das Gradientenverfahren mit einer Lernrate von gamma = 1:

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

def grad_f(x):
    return np.array([np.cos(x[0])+0.1*x[0], 0.2*x[1]])

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)
i = 1

In [ ]:
contour_plot.add_gradient_descent(x0=[-4, -2], function = f, grad=grad_f, gamma=1, Iterationen=i, color = "#636EFA")
contour_plot.show() # noch 3d-plot??
i += 1

### Das Gradientenverfahren mit einer Lernrate von $\gamma = 0.1$ ist langsamer:

In [ ]:
i = 1

In [ ]:
# Das Gradientenverfahren mit einer Lernrate von gamma = 0.1:

contour_plot.add_gradient_descent(x0=[-4, -2], function = f, grad=grad_f, gamma=0.1, Iterationen=i, color = "#EF553B")

contour_plot.show()

i+=1

### Eine Lernrate von $\gamma = 2$ führt hingegen nicht zu einer Konvergenz:

In [ ]:
i = 1

In [ ]:
# Das Gradientenverfahren mit einer Lernrate von gamma = 2:

contour_plot.add_gradient_descent(x0=[-4, -2], function = f, grad=grad_f, gamma=2, Iterationen=i, color = "#00CC96")

contour_plot.show() # noch 3d-plot??

i += 1

## Einfluss der Startwerts

Wenn die zu minimierende Funktion mehrere globale Minima besitzt, spielt die Wahl des Startwertes auch eine große Rolle: 

In [ ]:
import numpy as np
def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

contour_plot = plot()
contour_plot.plot_contour(-5,6,-3,3,f)

surface_plot = plot()
surface_plot.plot_surface(-5,6,-3,3,f)
surface_plot.show()

In [ ]:
import random

contour_plot.add_gradient_descent(x0=[random.uniform(-5,6),random.uniform(-3,3)], function = f, grad=grad_f, gamma=1, Iterationen=10)
contour_plot.show()

## Nichtlineare Optimierung mit Scipy

Das vorgestellte Verfahren ist für die Praxis noch zu einfach, in der praktischen Anwendung gibt es noch viele Probleme, die zu beachten sind.
Deswegen nimmt man normalerweise eine bereits vorhandene Implementierung.
Hierfür eignet sich das Python-Paket Scipy mit der Funktion *minimize* sehr gut.
Es reicht, die zu minimierende Funktion und den Startwert einzugeben:

In [ ]:
# Optimierung mit scipy
import numpy as np
from scipy.optimize import minimize

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

minimize(fun=f, x0=np.array([-4,-2]))

### Bessere Performance druch Eingabe des Gradienten

Wenn man zusätzlichn den Gradienten als Argument übergibt, konvergiert das Verfahren schneller:

In [ ]:
def grad_f(x):
    return np.array([np.cos(x[0])+0.1*x[0], 0.2*x[1]])

minimize(fun=f, x0=np.array([-4,-2]), jac=grad_f)

## Nebenbedingungen der Form $g(x) \leq 0$

Ein Beispiel für eine Nebenbedingung der Form $g(x) \leq 0$ ist
\begin{equation}
-(x+2)^{2} + y^{3} \leq 0
\end{equation}

Dies Definiert einen Bereich, in dem sich die Lösung befinden kann.

In [ ]:
def h(x):
    return (x[0]+3)**3-x[1]

def g_(x):
    return - ((x+2)**2)**(1/3)

x1 = np.linspace(-5, -2, 100).tolist()
y1 = [np.sqrt(np.abs(2.25 - (x+3.5)**2)) for x in x1]
x2 = np.linspace(-2, -5, 100).tolist()
y2 = [-np.sqrt(np.abs(2.25 - (x+3.5)**2)) for x in x2]
x = x1 + x2
y = y1 + y2

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)

contour_plot.add_trace(go.Scatter(x=x, y=y, fill='tonexty', showlegend = False, marker = {'color' : 'cyan'})).show()

## Nebenbedingungen der Form $h(x) = 0$

Eine Nebenbedingung der Form $h(x) = 0$ definiert überlicherweise eine Linie, auf der sich Lösung befindet.
Dies ist im allgemeinen ein aktive Bedingung, d.h. es wird ein schlechteres Ergebnis als ohne Nebenbedingung erreicht.
Wir zeigen als Beispiel eine Nebenbedingung der Form
\begin{equation} h(x, y) = (x + 3)^2 - y. \end{equation}

In [ ]:
contour_plot.add_h()
contour_plot.show()

## Das quadratische Penalty-Verfahren
Wir wissen jetzt, wie man unrestringierte nichtlineare Optimierungsprobleme lösen kann.
Doch wie können wir die Nebenbedingungen berücksichtigen, also

\begin{equation}
\begin{split}
\text{Minimiere }& f(x) \\
\text{wobei }& g(x) \leq 0 \\
\text{und }& h(x) = 0 \text{ gilt?}
\end{split}
\end{equation}

Eine Möglichkeit ist es, zu der zu minimierenden Funktion $f$ einen Strafterm hinzuzuaddieren, der Abweichungen von den Nebenbedingungen bestraft.
Solche Verfahren heißen **Penalty-Verfahren**.
Wir stellen hier das quadratische **Penalty-Verfahren** vor.
Das Problem lässt sich dann als

\begin{equation}
\text{Minimiere } f(x) + \tfrac{ \alpha}{2} max(0, g(x))^{2} + \tfrac{ \alpha}{2} h(x)^{2}
\end{equation}

formulieren.

## Das Penalty-Verfahren für $h(x) = 0$:

Zur Erinnerung: 

\begin{equation}h(x, y) = (x + 3)^2 - y.\end{equation}

Wir wählen ein $\alpha \in \mathbb{R}$ und müssen dann die Funktion

\begin{equation}
\begin{split}
f_{\text{pen}} (x, y)
&=
f(x, y) + \alpha h(x, y)^2
\end{split}
\end{equation}

minimieren.
Der Gradient von $h(x, y)^2$ ist

\begin{equation}
\nabla h^2(x,y) =
\begin{pmatrix}
4x^{3} + 36x^{2} + 108x + 108 - 4xy - 12y \\
-2x^{2} -12x + 2y - 18
\end{pmatrix}
\end{equation}

Der Gradient von $f_{ \text{pen}}$ ist

\begin{equation}
\nabla f_{ \text{pen}} = 
\nabla f + \alpha \nabla g.
\end{equation}

In [ ]:
alpha = 0.01

def h(x):
    return (x[0]+3)**2 - x[1]

def f_pen(x):
    return f(x) + alpha*h(x)**2

surface_plot = plot()
surface_plot.plot_surface(-5,1,-3,3,f,opacity = 0.2)
surface_plot.plot_surface(-5,1,-3,3,f_pen, opacity = 0.8, showscale=False, colorscale = 'oxy')
surface_plot.show()

contour_plot = plot()
contour_plot.plot_contour(-5,1,-3,3,f)
contour_plot.add_h()

def grad_h_sq(x):
    return np.array([4*x[0]**3 + 36*x[0]**2 + 108*x[0] + 108 - 4*x[0]*x[1] - 12*x[1],
                     -2*x[0]**2 - 12*x[0] + 2*x[1] - 18])

x0=[0, 3]

gamma = 1

In [ ]:
contour_plot.add_gradient_descent(x0=x0, function=f, grad=lambda x : (grad_f(x) + alpha*grad_h_sq(x)), gamma=gamma,
                                  Iterationen=100, Nebenbedingung = h)

contour_plot.show()

alpha *= 1.5
gamma /= 1.5
x0 = contour_plot.result

## Optimierung mit Nebenbedingungen in Python

Auch mit Nebenbedingungen kann man in Python gut optimieren.

### Optimierung mit $h(x, y) = 0$:

In [ ]:
# Optimierung mit scipy
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2

def h(x):
    return (x[0]+3)**2 - x[1]

constraints =NonlinearConstraint(h, lb = 0, ub = 0)

minimize(fun=f, x0=np.array([3,0]), constraints=constraints)

### Optimierung mit $h(x, y) = 0$ und $g(x, y) \leq 0$:

In [ ]:
def g(x):
    return -(x[0]+2)**2 + x[1]**3

constraints = [NonlinearConstraint(h, lb = 0, ub = 0),
               NonlinearConstraint(g, lb = -np.inf, ub = 0)]

minimize(fun=f, x0=np.array([-4,-2]), constraints=constraints)

## Ausblick: Automatic Differentiation

Optimierungsprobleme lassen sich oft besser lösen, wenn der Gradient der zu minimierenden Funktion bekannt ist.
In der Praxis sind die Ableitungen aber zu schwierig zu berechnen.
**Automatic differentation** schafft hier Abhilfe:
Entsprechende Pakete können von vielen in Python definierten Funktionen die mathematische Ableitung bestimmen (im Gegensatz zur numerischen Ableitung).
Dies ermöglicht auch eine effiziente Implementierung der Optimierung bei komplizierten Zielfunktionen, und eine bessere Anwendbarkeit.

In [ ]:
import autograd.numpy as np   # Thinly-wrapped version of Numpy
from autograd import grad, hessian
from scipy.optimize import minimize

def f(x):
    return np.sin(x[0]) + 0.05*x[0]**2 + 0.1*x[1]**2
    
grad_f = grad(f)

print(minimize(fun=f, method = 'SLSQP', x0=np.array([-4,-2])))

hess_f = hessian(f)

minimize(fun=f, jac = grad_f, hess = hess_f, method= 'Newton-CG', x0=np.array([-4,-2]))

# nit ist die Anzahl der Iterationen
# nfev : number of function evaluations

### Automatic differentiation verbessert auch das Konvergenzverhalten für die Optimierung mit Nebenbedingungen

In [ ]:
# Optimierung mit scipy
from scipy.optimize import NonlinearConstraint

def h(x):
    return (x[0]+3)**2 - x[1]
def g(x):
    return (-(x[0]+2)**2 + x[1]**3)

constraints = [NonlinearConstraint(h, lb = 0, ub = 0),
               NonlinearConstraint(g, lb = -np.inf, ub = 0)]
    
minimize(fun=f, x0=np.array([-4,-2]), constraints=constraints)

In [ ]:
grad_h = grad(h)

def hess_h(x, y):
    np.matmul(y, hessian(h)(x))
grad_g = grad(g)
#hess_h = hessian(h)
grad_f = grad(f)
hess_f = hessian(f)

#def hess_h(m):
#    return np.matmul(hess_h_, m)

def hess_g(m):
    return np.matmul(hess_h_, m)

constraints = [NonlinearConstraint(h, lb = 0, ub = 0, jac = grad_h),
               NonlinearConstraint(g, lb = -np.inf, ub = 0, jac = grad_g)]
    
minimize(fun=f, x0=np.array([-4,-2]), jac = grad_f, hess = hess_f, constraints=constraints)

